In [ ]:
import h5py
import os

h5_dir = "wsi_patches/BLCA/patches/"
h5_files = os.listdir(h5_dir)

# Open the H5 file in read mode
with h5py.File(
    "wsi_patches/BLCA/patches/TCGA-MV-A51V-01Z-00-DX1.5D626704-0803-4912-96D5-FB1EEFA509FB.h5",
    "r",
) as f:
    # Print the keys (top-level groups/datasets) in the file
    print("Keys in H5 file:", list(f.keys()))

    # Access a specific group or dataset
    # Replace 'your_group_or_dataset_key' with the actual key
    print(f["coords"][:])

SyntaxError: expected ':' (2076677357.py, line 8)

In [ ]:
# run this command in terminal
"python create_patches_fp.py --source DATA_DIRECTORY --save_dir RESULTS_DIRECTORY --patch_size 256 --seg --patch --stitch"

In [182]:
import openslide

wsi = openslide.open_slide(
    "wsi_files/BLCA/TCGA-MV-A51V-01Z-00-DX1.5D626704-0803-4912-96D5-FB1EEFA509FB.svs"
)
wsi.level_count

3

In [188]:
import matplotlib.pyplot as plt
from torchvision import transforms
width, height = 256, 256
full_image = wsi.read_region((1216, 6928), 0, (width, height))
full_image = full_image.convert("RGB")  # Convert RGBA to RGB
transform = transforms.Compose([
    transforms.Resize((width, height)),
    transforms.ToTensor(),
])
full_image = transform(full_image)
full_image.shape

torch.Size([3, 256, 256])

# Dataset

In [5]:
import os
import requests
import re

## Download data from GDC

In [6]:
import pandas as pd

txt_file_path = "gdc_manifest.2025-11-08.140931.txt"
manifest_file = pd.read_csv(txt_file_path, sep="\t")
manifest_file.head()

,id,filename,md5,size,state
0,c85e943d-6cca-4b5b-ab63-ad8ef320aefc,TCGA-BL-A13I-01Z-00-DX2.251412B0-EECB-420A-B96...,46bfa311930514a3d06f4deb89166c6a,552174543,released
1,03414f4a-2121-4272-be1d-62c587d1ed26,TCGA-CF-A47S-01Z-00-DX1.1397F7C0-1098-4C42-B15...,111d59a5cf8ff9b5e563523f8eec2a40,1231578754,released
2,67f49f95-8307-4157-b575-ec96bee225a8,TCGA-MV-A51V-01Z-00-DX1.5D626704-0803-4912-96D...,4216caeea91c5d8517239a6ddc434142,55379431,released
3,cec56b4e-1f3c-48d3-81b5-a7c15984949a,TCGA-GU-AATO-01Z-00-DX1.1F0F2098-E4DF-4AA3-819...,112e0e398cd46e439976d90dc99a8822,1558004613,released
4,b6bd14fc-7e2c-41d2-aae3-7d510b58c105,TCGA-DK-A1AF-01Z-00-DX1.E18DE029-27A6-43A6-9E9...,dbfcb97f5e1309b2d9131526e278bf0d,2271582867,released


In [7]:
# file_ids = [
#     "511e366e-de4b-49fc-a70d-99ccc4c464d5",
#     "37a099ab-14d1-41c1-aa40-c823bba83dce",
#     "c3b21f63-5c8a-4610-951f-ed62643fd355",
#     "5e4ef3f3-5617-45bc-95ee-13fd5c6d0767",
# ]
save_dir = "./wsi_files/BLCA"


def download_svs(file_id, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    data_endpt = "https://api.gdc.cancer.gov/data/{}".format(file_id)

    response = requests.get(
        data_endpt, headers={"Content-Type": "application/octet-stream"}
    )

    # The file name can be found in the header within the Content-Disposition key.
    response_head_cd = response.headers["Content-Disposition"]

    file_name = re.findall("filename=(.+)", response_head_cd)[0]

    with open(os.path.join(save_dir, file_name), "wb") as output_file:
        output_file.write(response.content)


# for file_id in file_ids:
#     download_svs(file_id, save_dir)

In [ ]:
from tqdm import tqdm

open("wsi_files/BLCA/saved_ids.txt", "w").close()
for id in tqdm(manifest_file["id"]):
    download_svs(id, save_dir)
    with open("wsi_files/BLCA/saved_ids.txt", "a") as file:
        file.write(id + "\n")

  0%|          | 1/457 [08:22<63:38:32, 502.44s/it]


KeyboardInterrupt: 

## Load dataset

In [84]:
import pandas as pd
import os

# read csv file
df = pd.read_csv("tcga_blca_all_clean.csv")
df["case_id"].value_counts()

case_id
TCGA-G2-AA3F    9
TCGA-G2-A2EJ    8
TCGA-G2-A2EO    8
TCGA-G2-A2ES    7
TCGA-FJ-A3Z7    6
               ..
TCGA-ZF-AA4T    1
TCGA-ZF-AA4U    1
TCGA-ZF-A9R3    1
TCGA-ZF-A9R4    1
TCGA-ZF-A9R5    1
Name: count, Length: 373, dtype: int64

In [2]:
df.columns

Index(['case_id', 'slide_id', 'site', 'is_female', 'oncotree_code', 'age',
       'survival_months', 'censorship', 'train', 'NDUFS5_cnv',
       ...
       'ZWINT_rnaseq', 'ZXDA_rnaseq', 'ZXDB_rnaseq', 'ZXDC_rnaseq',
       'ZYG11A_rnaseq', 'ZYG11B_rnaseq', 'ZYX_rnaseq', 'ZZEF1_rnaseq',
       'ZZZ3_rnaseq', 'TPTEP1_rnaseq'],
      dtype='object', length=20404)

In [3]:
df_filtered = df.drop(columns=["site", "oncotree_code", "train"])

In [4]:
# get the patients in file_ids
wsi_path = "./wsi_files/BLCA/"
slide_ids = [file_id for file_id in os.listdir(wsi_path) if file_id.endswith(".svs")]
case_ids = [slide_id.split("-01Z")[0] for slide_id in slide_ids]
df_filtered = df_filtered[df_filtered["slide_id"].isin(slide_ids)]
df_filtered[["survival_months", "censorship"]].reset_index(drop=True)

,survival_months,censorship
0,7.33,0
1,2.66,0
2,5.06,0
3,10.94,1
4,17.61,1
5,8.94,0
6,10.64,0
7,19.38,1
8,63.96,1
9,12.16,0


In [5]:
df_filtered = df_filtered.drop(columns=["slide_id"])

In [6]:
clinical_cols = ["survival_months", "censorship"]
feature_cols = [col for col in df_filtered.columns if col not in clinical_cols]

In [7]:
# find categorical and numerical columns
categorical_cols = ["is_female"]
numeric_cols = []
for col in feature_cols:
    if col not in categorical_cols:
        numeric_cols.append(col) 

In [8]:
df1 = df_filtered[df_filtered["case_id"] == case_ids[0]].iloc[0]
df1 = df1[feature_cols]

In [9]:
from sklearn.model_selection import train_test_split

X = df_filtered[feature_cols]
y = df_filtered[clinical_cols]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train.shape, X_test.shape

((12, 20398), (3, 20398))

In [10]:
X_train.columns

Index(['case_id', 'is_female', 'age', 'NDUFS5_cnv', 'MACF1_cnv',
       'RNA5SP44_cnv', 'KIAA0754_cnv', 'BMP8A_cnv', 'PABPC4_cnv',
       'SNORA55_cnv',
       ...
       'ZWINT_rnaseq', 'ZXDA_rnaseq', 'ZXDB_rnaseq', 'ZXDC_rnaseq',
       'ZYG11A_rnaseq', 'ZYG11B_rnaseq', 'ZYX_rnaseq', 'ZZEF1_rnaseq',
       'ZZZ3_rnaseq', 'TPTEP1_rnaseq'],
      dtype='object', length=20398)

In [11]:
# from torch.utils.data import Dataset
# import torch
# import openslide
# import h5py
# from torchvision import transforms

# class PatientDataset(Dataset):
#     def __init__(self, case_ids: list[str], h5_files: list[str], X, y):
#         self.case_ids = case_ids
#         self.h5_files = h5_files
#         self.X = X
#         self.y = y
#         self.patches = []
#         self.clinical_cols = ["survival_months", "censorship"]
#         self.feature_cols = [
#             col for col in self.X.columns if col not in self.clinical_cols
#         ]
#         self.feature_cols.remove("case_id")
#         for h5_file in h5_files:
#             assert os.path.exists(h5_file), f"H5 file {h5_file} does not exist."
#         for case_id in case_ids:
#             for h5_file in h5_files:
#                 if case_id not in h5_file:
#                     continue
#                 data = h5py.File(h5_file, "r")
#                 wsi = openslide.open_slide(
#                     f"wsi_files/BLCA/{h5_file.split('/')[-1].split('.h5')[0]}.svs"
#                 )
#                 patches = []
#                 for coord in data["coords"][:]:
#                     patch = wsi.read_region(coord, 2, (256, 256)).convert("RGB")
#                     tensor_patch = transforms.ToTensor()(patch)
#                     patches.append(tensor_patch)
#                 self.patches.append(patches)

#     def __len__(self):
#         return len(self.case_ids)

#     def __getitem__(self, idx):
#         patches = self.patches[idx]  # patches for one patient
#         case_id = self.case_ids[idx]
#         patient = self.X[self.X["case_id"] == case_id].iloc[0]
#         patient = patient[self.feature_cols]
#         patient = torch.tensor(patient.values.astype(float))
#         return (patient, patches), torch.tensor(self.y.iloc[idx].values.astype(float))

In [12]:
from torch.utils.data import Dataset
import torch
import openslide
import h5py
from torchvision import transforms


class PatientDataset(Dataset):
    def __init__(self, case_ids: list[str], h5_files: list[str], X, y):
        self.case_ids = case_ids
        self.h5_files = h5_files
        self.X = X
        self.y = y
        self.patches = []
        self.clinical_cols = ["survival_months", "censorship"]
        self.feature_cols = [
            col for col in self.X.columns if col not in self.clinical_cols
        ]
        self.feature_cols.remove("case_id")
        self.patch_features = []
        for h5_file in h5_files:
            assert os.path.exists(h5_file), f"H5 file {h5_file} does not exist."
        for case_id in case_ids:
            # load tensor from extracted features
            extracted_path = f"wsi_patches/BLCA/features/{case_id}.pt"
            features = torch.load(extracted_path)
            self.patch_features.append(features)

    def __len__(self):
        return len(self.case_ids)

    def __getitem__(self, idx):
        patch_features = self.patch_features[idx]
        case_id = self.case_ids[idx]
        patient = self.X[self.X["case_id"] == case_id].iloc[0]
        patient = patient[self.feature_cols]
        patient = torch.tensor(patient.values.astype(float))
        return (patient, patch_features), torch.tensor(self.y.iloc[idx].values.astype(float))

# Neural network

In [13]:
import torch.nn as nn
import torchvision.models as models
import torch

In [14]:
class PathologicalEncoder(nn.Module):
    def __init__(self, hidden_dim: int = 128):
        super(PathologicalEncoder, self).__init__()
        self.fc = nn.LazyLinear(hidden_dim)

    def forward(self, x):
        # x shape: 
        x = self.fc(x)
        return x

For each image $i$, the number patches per image is: $M_i$. After computing self-attention, the shape is: $M_i \times d$. After that, we use the mean pooling to summarize the attention into an image feature vector of size $d$

In [15]:
class BaseGenomicEncoder(nn.Module):
    def __init__(
        self, df: pd.DataFrame, categorical_cols: list[str], numeric_cols: list[str]
    ):
        super(BaseGenomicEncoder, self).__init__()
        self.categorical_cols = categorical_cols
        self.numerical_cols = numeric_cols
        self.features = self.numerical_cols + self.categorical_cols
        self.embeddings = nn.ModuleDict()
        for col in self.categorical_cols:
            num_unique_values = int(df[col].nunique())
            embedding_size = 4
            self.embeddings[col] = nn.Embedding(num_unique_values, embedding_size)

    def categorical_name_to_index(self, col):
        return self.categorical_cols.index(col)

    def numerical_name_to_index(self, col):
        return self.numerical_cols.index(col)

    def categorical_index_to_name(self, index):
        return self.categorical_cols[index]

    def numerical_index_to_name(self, index):
        return self.numerical_cols[index]

    def embed(self, x):
        embedded_cols = []
        for col in self.categorical_cols:
            # ndarray = np.array()
            embedded_col = self.embeddings[col](
                x[:, self.categorical_name_to_index(col)].long()
            )
            # print(embedded_col.shape)
            embedded_cols.append(embedded_col)
        numerical_data = torch.stack(
            [x[:, self.numerical_name_to_index(col)] for col in self.numerical_cols],
            dim=1,
        ).float()
        x = torch.cat(embedded_cols + [numerical_data], dim=1)
        return x

    def embed_with_time(self, x, t):
        time_data = torch.reshape(t, (x.shape[0], 1)).float()
        x = self.embed(x)
        x = torch.cat((x, time_data), dim=1)
        return x

    def forward(self, x):
        raise NotImplementedError("Forward method not implemented!")


class GenomicEncoder(BaseGenomicEncoder):
    def __init__(
        self,
        df: pd.DataFrame,
        categorical_cols: list[str],
        numeric_cols: list[str],
        bias=True,
        hidden_dim: int = 128,
    ):
        super().__init__(df, categorical_cols, numeric_cols)

        self.net = nn.Sequential(
            nn.LazyLinear(128),
            nn.LeakyReLU(0.1),
            nn.LazyLinear(256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),
            nn.LazyLinear(128),
        )
        self.fc = nn.LazyLinear(hidden_dim)

        # self.bias = nn.Sequential(nn.LazyLinear(32), nn.ReLU(), nn.LazyLinear(1))

    def forward(self, x):
        x = self.embed(x)
        x = self.net(x)
        x = self.fc(x)
        return x.squeeze()


# loss
class NLL(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, preds, failure_times, is_observed):
        # Number of observed events
        return (
            1
            / len(preds)
            * torch.sum(torch.exp(preds) * failure_times - is_observed * preds, dim=0)
        )

In [16]:
class SurvivalModel(nn.Module):
    def __init__(
        self,
        path_encoder: PathologicalEncoder,
        geno_encoder: GenomicEncoder,
        hidden_dim: int = 128,
    ):
        super(SurvivalModel, self).__init__()
        self.path_encoder = path_encoder
        self.geno_encoder = geno_encoder
        self.fc = nn.LazyLinear(1)
        self.hidden_dim = hidden_dim
        self.path_msa = nn.MultiheadAttention(
            embed_dim=hidden_dim, num_heads=4, batch_first=True
        )
        self.geno_msa = nn.MultiheadAttention(
            embed_dim=hidden_dim, num_heads=4, batch_first=True
        )

    def forward(self, path_x, geno_x, mask):
        # path_x shape: Batch size x #patches x Feature dim
        # flatten:
        B, N, D = path_x.shape
        path_x = path_x.view(-1, D)  # shape: (Batch size * Num patches) x Feature dim
        # geno_x shape: Batch size x Num genomic features
        path_features = self.path_encoder(
            path_x
        )  # shape: (Batch size * Num patches) x Feature dim
        path_features = path_features.view(
            B, N, -1
        )  # shape: Batch size x Num patches x Feature dim
        geno_features = self.geno_encoder(geno_x)  # shape: Batch size x Feature dim

        # Self Attention Mechanism
        path_attended, _ = self.path_msa(
            path_features, path_features, path_features, key_padding_mask=mask
        )  # shape: Batch size x Num patches x Feature dim
        path_representation = torch.mean(
            path_attended, dim=1
        )  # shape: Batch size x Feature dim
        geno_features = geno_features.view(B, -1)  # shape: Batch size x Feature dim
        # concat path and genomic features
        combined_features = torch.cat(
            (path_representation, geno_features), dim=1
        )  # shape: Batch size x (2 * Feature dim)
        preds = self.fc(combined_features)  # shape: Batch size x 1
        return preds.squeeze()  # shape: Batch size

# Run model

In [17]:
h5_dir = "wsi_patches/BLCA/patches/"
h5_files = os.listdir(h5_dir)

case_ids = [h5_file.split("-01Z")[0] for h5_file in h5_files]

In [18]:
X = df_filtered[feature_cols]
y = df_filtered[clinical_cols]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [19]:
train_case_ids = X_train["case_id"].to_list()
test_case_ids = X_test["case_id"].to_list()
train_h5_files = [
    os.path.join(h5_dir, h5_file)
    for h5_file in h5_files
    if h5_file.split("-01Z")[0] in train_case_ids
]
test_h5_files = [
    os.path.join(h5_dir, h5_file)
    for h5_file in h5_files
    if h5_file.split("-01Z")[0] in test_case_ids
]

In [24]:
from torch.nn.utils.rnn import pad_sequence


def collate_fn(batch):
    patients, patches, clinical_outcomes = [], [], []
    for (patient, image_patches), clinical_outcome in batch:
        patients.append(patient)
        patches.append(image_patches)
        clinical_outcomes.append(clinical_outcome)
    max_num_patches = max([patch.shape[0] for patch in patches])
    mask = torch.zeros(len(patches), max_num_patches, dtype=torch.bool)
    for i, patch in enumerate(patches):
        mask[i, : patch.shape[0]] = True

    patients = torch.stack(patients)
    patches = pad_sequence(patches, batch_first=True)

    clinical_outcomes = torch.stack(clinical_outcomes)
    return patients, patches, clinical_outcomes, mask

In [21]:
train_dataset = PatientDataset(
    train_case_ids,
    train_h5_files,
    X_train.reset_index(drop=True),
    y_train.reset_index(drop=True),
)
test_dataset = PatientDataset(
    test_case_ids,
    test_h5_files,
    X_test.reset_index(drop=True),
    y_test.reset_index(drop=True),
)

In [68]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn
)

In [69]:
patients, patches, clinical_outcomes, mask = next(iter(train_loader))

In [70]:
geno_encoder = GenomicEncoder(
    df=df_filtered,
    categorical_cols=categorical_cols,
    numeric_cols=numeric_cols,
    hidden_dim=128,
)
geno_encoder(patients)

tensor([[-0.0267,  0.0806,  0.1579,  ...,  0.0932, -0.0733, -0.0191],
        [-0.1879,  0.5568,  0.4099,  ...,  1.1018,  0.2168,  0.7740],
        [-0.0252,  0.1402,  0.1855,  ...,  0.1989,  0.0407,  0.0255],
        ...,
        [ 0.0136, -0.0237,  0.1997,  ...,  0.0925, -0.0893,  0.0490],
        [-0.0164,  0.1039,  0.1222,  ...,  0.1967, -0.0741,  0.0221],
        [-0.2093,  0.1398,  0.1696,  ..., -0.0502,  0.1029, -0.0734]],
       grad_fn=<SqueezeBackward0>)

In [71]:
patients.shape, patches.shape, clinical_outcomes.shape, mask.shape

(torch.Size([8, 20397]),
 torch.Size([8, 413, 2048]),
 torch.Size([8, 2]),
 torch.Size([8, 413]))

In [72]:
# get index of categorical and numerical columns
categorical_cols_idx = [feature_cols.index(col) for col in categorical_cols]
numeric_cols_idx = [feature_cols.index(col) for col in numeric_cols]

In [73]:
path_enc = PathologicalEncoder(hidden_dim=128)
geno_enc = GenomicEncoder(
    df_filtered,
    categorical_cols=categorical_cols,
    numeric_cols=numeric_cols,
    hidden_dim=128,
)
survival_model = SurvivalModel(
    path_encoder=path_enc,
    geno_encoder=geno_enc,
    hidden_dim=128,
)
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
survival_model = survival_model.to(device)

In [74]:
from torch.optim import Adam

lr = 0.0005
optimizer = Adam(survival_model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
loss = NLL()
num_epochs = 10

In [75]:
def train(model, optimizer, loss, num_epochs, device):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for patient, patches, clinical_outcomes, mask in train_loader:
            optimizer.zero_grad()
            patches = patches.to(device)
            patient = patient.to(device)
            clinical_outcomes = clinical_outcomes.to(device)
            mask = ~mask.to(device)  # invert mask for key_padding_mask
            preds = model(patches, patient, mask)
            failure_times = clinical_outcomes[:, 0]
            is_observed = clinical_outcomes[:, 1]
            current_loss = loss(preds, failure_times, is_observed)
            current_loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += current_loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

In [76]:
torch.cuda.empty_cache()

In [77]:
train(survival_model, optimizer, loss, num_epochs, device)

Epoch 1/10, Loss: 20.3452
Epoch 2/10, Loss: 3.5865
Epoch 3/10, Loss: 2.7761
Epoch 4/10, Loss: 3.0143
Epoch 5/10, Loss: 3.8023
Epoch 6/10, Loss: 4.2184
Epoch 7/10, Loss: 3.4105
Epoch 8/10, Loss: 3.7831
Epoch 9/10, Loss: 2.7460
Epoch 10/10, Loss: 3.4222


In [78]:
from lib.metrics import c_index

train_loader = DataLoader(
    train_dataset, batch_size=len(train_dataset), shuffle=False, collate_fn=collate_fn
)
survival_model.eval()
collected_preds = []
clinical_outcomes_list = []
with torch.no_grad():
    for patient, patches, clinical_outcomes, mask in train_loader:
        patches = patches.to(device)
        patient = patient.to(device)
        clinical_outcomes = clinical_outcomes.to(device)
        mask = ~mask.to(device)  # invert mask for key_padding_mask
        preds = survival_model(patches, patient, mask)
        collected_preds.append(preds.cpu())
        clinical_outcomes_list.append(clinical_outcomes.cpu())

In [79]:
clinical_outcomes_list = torch.stack(clinical_outcomes_list)
clinical_outcomes_list = clinical_outcomes_list.squeeze(dim=1)

In [80]:
collected_preds = torch.stack(collected_preds)

In [81]:
clinical_outcomes_list = clinical_outcomes_list.squeeze()

In [82]:
clinical_outcomes_list.shape, collected_preds.shape

(torch.Size([12, 2]), torch.Size([1, 12]))

In [83]:
train_c_index_value = c_index(
    collected_preds, clinical_outcomes_list[:, 0], clinical_outcomes_list[:, 1]
)
print(f"Train C-index: {train_c_index_value:.4f}")

Train C-index: 0.9200
